In [64]:
import pandas as pd
import pandasql as ps

In [70]:
def name_format(name):
    temp = name.strip().split(" ")
    new = " ".join(t for t in temp if t)
    return new

bio = pd.read_csv("all_players_clean.csv")
ws = pd.read_csv("./ws/all_winshare.csv")

bio["Name"] = bio["Name"].apply(name_format)
ws["Player"] = ws["Player"].apply(name_format)

In [71]:
bio_ws = pd.merge(bio, ws, how='left', left_on=['Name', 'Year'], right_on=['Player', 'Year'])
bio_ws.drop(columns='Player', inplace=True)
bio_ws.to_csv("bio_ws.csv", index=False)

In [69]:
record = pd.read_csv("record.csv")
# record['Year'] = record['Year'].str.slice(0, 4)
# record["conf_win_rate"] = record["Conference Win"] / (record["Conference Win"] + record["Conference Lose"])
# record.to_csv("record.csv", index=False)\
record['Team'] = record['Team'].str.replace(" ", "_")
record[:3]

,Team,Year,Conference Win,Conference Lose,Overall Win,Overall Lose,conf_win_rate
0,Illinois,2021,15,5,23,10,0.75
1,Illinois,2022,11,9,20,13,0.55
2,Illinois,2023,14,6,29,9,0.70


In [51]:
bio_team = ps.sqldf("""
    SELECT School, Year, COUNT(CASE WHEN Is_Transfer = True THEN 1 END) AS transfer_amount, 
        AVG(Height) as mean_height, AVG(Weight) as mean_weight, AVG(Age) as mean_age
    FROM bio_ws
    GROUP BY School, Year
""")
bio_team[:3]

,School,Year,transfer_amount,mean_height,mean_weight,mean_age
0,Illinois,2020,0,78.176471,207.941176,20.000000
1,Illinois,2021,2,78.750000,215.625000,20.250000
2,Illinois,2022,2,78.357143,204.642857,19.571429


In [63]:
bio_record = pd.merge(bio_team, record, how='right', left_on=['School', 'Year'], right_on=['Team', 'Year'])
bio_record.drop(columns='Team', inplace=True)
bio_record = bio_record.loc[:, ~bio_record.columns.str.contains('^Unnamed')]
bio_record.to_csv("bio_record.csv", index=False, encoding='utf-8')